# Dataset Matching

## 줌마영어 교재 데이터
- 줌마 영어 교재 챕터 당 여러상황이 매치됨
- 일대일 관계 -> 일대다 변형 필요
### Data Summary
항목 | 값
---|--
교재 챕터 수 | 19
대화 턴 | 12~14
생성 가능 Persona | 19
생성 가능 History | 11~13



In [1]:
import pandas as pd
import numpy as np
import json
from itertools import islice, chain, cycle
import random
import copy
import contractions
import spacy

In [2]:
spacy.prefer_gpu()
spacy.require_gpu()

True

In [3]:
zumma_book = pd.read_excel('data/zumma_engbook_persona.xlsx')
zumma_book.head(10)

,situation,persona
0,immigration checkpoint,I have a pasport.
1,immigration checkpoint,I am first visit in Singapore.
2,immigration checkpoint,I will be staying at the hotel.
3,immigration checkpoint,I will be staying for three days.
4,In a Taxi,I'd like to go to the hotel.
5,In a Taxi,It is 20 minutes from here.
6,In a Taxi,We will be there soon.
7,In a Taxi,it's 19 dollars.
8,Hotel Check-in,My name is Zoomma.
9,Hotel Check-in,I want to check in.


## AI HUB 상황 데이터

### Data Summary
항목 | 값
---|--
상황 수 | 2,769
대화 수  | 100,000
최대 생성 Persona | 2,769
최대 생성 History | 25,000


In [4]:
ai_hub = pd.read_excel('data/dialog_personaen(en).xlsx', index_col=0)
ai_hub['situation'] = ai_hub['situation'].str.replace(' / ', '/')
ai_hub.head(10)

,major,minor,situation,scenario,en
0,비즈니스,meeting,의견 교환하기,1,How is the market's reaction to the newly rele...
1,비즈니스,meeting,의견 교환하기,1,The sales increase is faster than the previous...
2,비즈니스,meeting,의견 교환하기,1,"Then, we'll have to call the manufacturer and ..."
3,비즈니스,meeting,의견 교환하기,1,"Sure, I'll make a call and double the volume o..."
4,비즈니스,meeting,의견 교환하기,2,Shall we take a look at the issues we discusse...
5,비즈니스,meeting,의견 교환하기,2,I believe that this week's new issues are more...
6,비즈니스,meeting,의견 교환하기,2,"Then, let's begin our meeting with the new iss..."
7,비즈니스,meeting,의견 교환하기,2,"Sure, the related materials are ready in front..."
8,비즈니스,meeting,의견 교환하기,3,"Do you mean we need to order additional 2,000 ..."
9,비즈니스,meeting,의견 교환하기,3,"Yes, time is running short, but we can manage it."


### 데이터 전처리
#### 소문자, 쉼표, 마췸표 구분

In [5]:
nlp = spacy.load('en_core_web_sm')
def spacingPunct(text):
    sentence = [token.text for token in nlp(text)]
    return " ".join(sentence)

def preprocessing(df, column_name):
    df[column_name] = df[column_name].str.lower()
#     df[column_name] = df[column_name].apply(lambda x : spacingPunct(x))
    return df[column_name]

def del_duplication(data_list):
    data_list  = list(data_list)
    data_list  = set(data_list)
    return list(data_list)

def find_matched_situation(situations):  #Zumma Text Book과 Ai Hub 상황 매칭(dialog)
    dialog = [ai_hub[ai_hub['situation']==situation]['en'] for situation in situations]
    dialog = pd.concat(dialog, axis=0).tolist()
    dialog = [dialog[i * 4:(i + 1) * 4] for i in range((len(dialog) + 4 - 1) // 4 )] #4개의 대화 셋이 하나의 Dialog
    return dialog


In [6]:
zumma_book['persona'] = preprocessing(zumma_book, 'persona')
zumma_book['situation'] = preprocessing(zumma_book, 'situation')

In [7]:
ai_hub['en'] = preprocessing(ai_hub, 'en')

In [8]:
ai_hub.iloc[0]['en']

"how is the market's reaction to the newly released product?"

### 상황 데이터

In [9]:
situations = del_duplication(zumma_book['situation'].tolist())
situations

['checking in at the airport',
 'renting a car',
 'asking for directions',
 'making a hotel reservation',
 'immigration checkpoint',
 'getting a souvenir',
 'in a hospital',
 'hotel check-out',
 'room service',
 'buying a camera',
 'in a taxi',
 'hotel check-in',
 'at a supermarket',
 'currency exchange',
 'asking someone to take a photo',
 'getting a dessert',
 'at a restaurant',
 'in flight',
 'at a shopping mall']

### 페스소나 데이터

In [10]:
persona_data= [{situation: zumma_book[zumma_book['situation']==situation]['persona'].tolist()} for situation in situations]
persona_data[0]

{'checking in at the airport': ['i have ticket and passport.',
  'i have one bag.',
  'i have one small bags with me.',
  'i would like a window sea.']}

## 줌마영어 교재(Persona dataset)과 AI HUB Dialog dataset 결합
- 줌마영어 교제의 dialogset에는 AI HUB데이터의 여러 상황이 대응됨.
- ex) in a hospital -> 증상을 묻고 답하는 상황,..,
- 이를 활용하여 Persona Dataset 제작
- 같은 Persona 중복을 피하기 위해 위치 변환을 통해 Persona를 뒤섞음(Hugging Face에서도 데이터 수를 augment하기 위해서 이 방법을 사용)

In [11]:
print(situations)

['checking in at the airport', 'renting a car', 'asking for directions', 'making a hotel reservation', 'immigration checkpoint', 'getting a souvenir', 'in a hospital', 'hotel check-out', 'room service', 'buying a camera', 'in a taxi', 'hotel check-in', 'at a supermarket', 'currency exchange', 'asking someone to take a photo', 'getting a dessert', 'at a restaurant', 'in flight', 'at a shopping mall']


In [12]:
#Persona  = [AI HUB Situationn] 매칭
situation_matched = {"in a hospital" : ["증상을 묻고 답하는 상황"],
                    'room service': ["룸서비스, 호텔 서비스, 불만 표시, 요청하기"],
                     'getting a souvenir' : ["기념품을 구매하는 상황", "여행 상품을 구매하는 상황", "기념품상점에서 기념품 구매하는 상황"],
                     'getting a dessert' : ["식사/디저트 주문" ],
                     'hotel check-out' : ["체크인/체크아웃 할 때"],
                     'making a hotel reservation' : ["호텔 (체크인, 항의, 부대시설 문의, 가격 문의)"],
                     'at a supermarket' : ["마트 (물건 찾기, 즉석/신선 제품)", "시장에서 물건사기",  ],
                     'at a shopping mall': ["아울렛, 리퍼브(반품) 매장(특별매장)에서 상품을 구매하고 문의 할때","옷가게, 신발가게 (사이즈, 착용, 탈의실 등)", "착용/사용 요청할 때", "할인 상품의 위치 및 가격에 대해 묻고 답하는 상황",
                                           ],
                     'hotel check-in' : ["체크인/체크아웃 할 때"],
                     'checking in at the airport': ["공항에서 체크인할 때"],
                     'at a restaurant': ["예약,입구,좌석 잡기", "메뉴/주문 접수 요청", "메인 식사 주문, 음료/후식 주문", "메뉴 주문", ], #예약,입구,좌석 잡기- 제거
                     'currency exchange' : ["환전 및 여행자 수표가 필요한 상황", "공항/은행에서 환전하는 상황", "환전을 하는 상황", ],
                     'in flight' : ["비행기 안 상황", ],
                     'in a taxi': ["택시 이용(요금문제, 목적지 등)", "택시 (찾기, 승차, 도착 후, 요금 지불)"], #택시 이용 추가( 더있음)
                     'renting a car': ["차를 렌트할 때 필수 조건에 대해 대화하는 상황", "렌트할 차를 고르는 상황", "렌트할 차의 요금에 대해 물어보는 상황"],
                     'asking for directions': ["여행 와서 길 물을 때", "여행자 안내소에서 안내가 필요한 상황 (길 묻기, 대중교통 시간표 등)", "길 잃은 상황" ],
                     'immigration checkpoint': ["입국심사 할 때", "입국/출국 상황( 입국심사, 수하물, 보안검색 )" ],
                     'buying a camera': ["아울렛, 리퍼브(반품) 매장(특별매장)에서 상품을 구매하고 문의 할때" ],
                     'asking someone to take a photo' : ['사진 촬영하고 싶을 때 다른 관광객에게 부탁']
                     }

In [13]:
situation_matched = sorted(situation_matched.items())

In [14]:
situation_matched

[('asking for directions',
  ['여행 와서 길 물을 때', '여행자 안내소에서 안내가 필요한 상황 (길 묻기, 대중교통 시간표 등)', '길 잃은 상황']),
 ('asking someone to take a photo', ['사진 촬영하고 싶을 때 다른 관광객에게 부탁']),
 ('at a restaurant',
  ['예약,입구,좌석 잡기', '메뉴/주문 접수 요청', '메인 식사 주문, 음료/후식 주문', '메뉴 주문']),
 ('at a shopping mall',
  ['아울렛, 리퍼브(반품) 매장(특별매장)에서 상품을 구매하고 문의 할때',
   '옷가게, 신발가게 (사이즈, 착용, 탈의실 등)',
   '착용/사용 요청할 때',
   '할인 상품의 위치 및 가격에 대해 묻고 답하는 상황']),
 ('at a supermarket', ['마트 (물건 찾기, 즉석/신선 제품)', '시장에서 물건사기']),
 ('buying a camera', ['아울렛, 리퍼브(반품) 매장(특별매장)에서 상품을 구매하고 문의 할때']),
 ('checking in at the airport', ['공항에서 체크인할 때']),
 ('currency exchange',
  ['환전 및 여행자 수표가 필요한 상황', '공항/은행에서 환전하는 상황', '환전을 하는 상황']),
 ('getting a dessert', ['식사/디저트 주문']),
 ('getting a souvenir',
  ['기념품을 구매하는 상황', '여행 상품을 구매하는 상황', '기념품상점에서 기념품 구매하는 상황']),
 ('hotel check-in', ['체크인/체크아웃 할 때']),
 ('hotel check-out', ['체크인/체크아웃 할 때']),
 ('immigration checkpoint', ['입국심사 할 때', '입국/출국 상황( 입국심사, 수하물, 보안검색 )']),
 ('in a hospital', ['증상을 묻고 답하는 상황']),
 ('in a t

In [15]:
#situation_dialog = [{situation :find_matched_situation(detail_situations)} for index, (situation, detail_situations) in enumerate(situation_matched)]  

In [16]:
situation_dialog ={}
for index, (situation, detail_situations) in enumerate(situation_matched):
    temp = {situation :find_matched_situation(detail_situations)}
    situation_dialog.update(temp)

In [17]:
situation_dialog['asking for directions'][0]

['excuse me. how can i get to this restaurant?',
 "it's very close. go straight this way and you'll see a big yellow sign. the restaurant is in the alley right next to it.",
 'thanks, how long will it take to get there?',
 'less than 5 minutes probably. keep walking straight until you see the yellow sign.']

#### 4개씩(dialog) 묶긴 상황 데이터
{'asking for directions': [['excuse me . how can i get to this restaurant ?',
   'its very close . go straight this way and you will see a big yellow sign . the restaurant is in the alley right next to it .',
   'thanks , how long will it take to get there ?',
   'less than 5 minutes probably . keep walking straight until you see the yellow sign .'],
  ['im trying to go to busan tomorrow . what time does the bus operate ?',
   'there are buses starting at 5 in the morning to busan .',
   'could i get tickets for two adults and one child for the 5 a .m . bus to busan tomorrow ?',
   'yes , for two adults and one child , the price will be 60 ,000 won .'],
  ['sorry , but is there a place in the park where i can take a bus ?',
   'yes , there is a bus just for going around the park .',
   'how often does the bus being operated for the park come ?',
   'the bus specifically for the park comes every 15 to 20 minutes .'],
  ['do i have to pay for the park pamphlet or map ?',

In [18]:
persona_data[0] #상황 : [페스소나,...,]

{'checking in at the airport': ['i have ticket and passport.',
  'i have one bag.',
  'i have one small bags with me.',
  'i would like a window sea.']}

# Utterance
#### History와 Candidate로 구성

In [19]:
def addingCandidate(next_sentence):
    candidate = copy.deepcopy(random.sample(ai_hub['en'].tolist(),19))
#     if next_sentence == None:
#         next_sentence = random.sample(ai_hub['en'].tolist(),1).copy()
    candidate.append(next_sentence)
    return candidate

In [20]:
def set_utterance(dialog):
    dialog.append("empty sentene")
    dialog.append("empty sentene")
    utterance = [{'candidates': addingCandidate(dialog[i+1]), 'history': dialog[:i+1]} for i,sentence in islice(enumerate(dialog), None,5,2)]
    return utterance

## Personality
#### Utterance와 Personality(Persona) 결합

In [21]:
data = []
for situation in situations:
    personality = zumma_book[zumma_book['situation']==situation]['persona'].tolist()
    dialogues = situation_dialog[situation].copy()
    for dialogue in dialogues:
        temp_person =[]
        np.random.shuffle(personality)
        temp_person = copy.deepcopy(personality)
        data.append(copy.copy({'personality': temp_person, 'utterances': set_utterance(dialogue)}))

## JSON Format

In [22]:
np.random.shuffle(data)
train_data = copy.deepcopy(data)
valid_data = np.random.choice(data, 200, replace=False).tolist()

In [24]:
train_val = {"train": train_data, "valid": valid_data}

In [25]:
with open('data/en_book_conversational.json', 'w', encoding="utf-8") as make_file:
    json.dump(train_val, make_file, ensure_ascii=False)